In [10]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

dataset = load_dataset("HuggingFaceTB/smoltalk", "all")

model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
# model_name = "Qwen/Qwen3-0.6B"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to('cpu')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
print(tokenizer.chat_template)
# tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
tokenizer.pad_token = tokenizer.eos_token

print(tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing well, thank you!"},
    ],
    tokenize=False,
))

del model, dataset


{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Hello, how are you?<|im_end|>
<|im_start|>assistant
I'm doing well, thank you!<|im_end|>



In [12]:
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    logging_steps=1,
    save_steps=100,
    bf16=False,
    fp16=False,
    assistant_only_loss=False,
    completion_only_loss=False,
)

trainer = SFTTrainer(
    args=training_args,
    model="HuggingFaceTB/SmolLM2-135M",
    processing_class=tokenizer,
    train_dataset=load_dataset("banghua/DL-SFT-Dataset")["train"],
)
trainer.train()

Truncating train dataset: 100%|██████████| 2961/2961 [00:00<00:00, 128791.19 examples/s]


Step,Training Loss
1,3.404800
2,3.015000


KeyboardInterrupt: 

In [16]:
dataset = load_dataset("banghua/DL-SFT-Dataset")
print(dataset["train"][0])

{'messages': [{'content': "- The left child should have a value less than the parent node's value, and the right child should have a value greater than the parent node's value.", 'role': 'user'}, {'content': "This statement is correct. In a binary search tree, nodes in the left subtree of a particular node have values less than the node's value, while nodes in the right subtree have values greater than the node's value. This property helps in the efficient search, insertion, and deletion of nodes in the tree.", 'role': 'assistant'}]}


In [22]:
from datasets import load_dataset

train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
print(train_dataset.select(range(10)))

dpo_ds = load_dataset("banghua/DL-DPO-Dataset", split="train")
print(dpo_ds.select(range(10)))

dataset = load_dataset("trl-lib/tldr", split="train")
print(dataset.select(range(10)))

Dataset({
    features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
    num_rows: 10
})
Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 10
})


c:\Users\leo.breedt\miniconda3\envs\neptune_scale_py_312_base\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leo.breedt\.cache\huggingface\hub\datasets--trl-lib--tldr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 6553/6553 [00:00<00:00, 303423.06 examples/s]


Dataset({
    features: ['prompt', 'completion'],
    num_rows: 10
})
